In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os


c:\Users\tonyl\Documents\Multimodal-Manga-Translator\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name_or_path = "tencent/HY-MT1.5-1.8B"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto")  # You may want to use bfloat16 and/or move to GPU here

Unrecognized keys in `rope_parameters` for 'rope_type'='dynamic': {'rope_theta', 'beta_slow', 'beta_fast', 'mscale', 'alpha', 'mscale_all_dim'}
Unrecognized keys in `rope_parameters` for 'rope_type'='dynamic': {'rope_theta', 'beta_slow', 'beta_fast', 'mscale', 'alpha', 'mscale_all_dim'}
Loading weights: 100%|██████████| 355/355 [00:00<00:00, 1000.52it/s, Materializing param=model.norm.weight]                               
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [ ]:
# model.save_pretrained("/backend/models/TencentHY/model")
# tokenizer.save_pretrained("/backend/models/TencentHY/tokenizer")

Writing model shards: 100%|██████████| 1/1 [00:04<00:00,  4.51s/it]


('./TencentHY/tokenizer\\tokenizer_config.json',
 './TencentHY/tokenizer\\chat_template.jinja',
 './TencentHY/tokenizer\\tokenizer.json')

In [19]:
ocr_text = """
    加油！你可以的！

    你是不是在吃醋？

    给我个面子，别再说了。

    你这个人真的很二。

    简直是自寻死路！

    你找死
"""

In [ ]:
messages = [
    {"role": "system", "content": "You are a professional Manhua translator. Translate dialogue into natural, punchy English used in action manga. Use genre-appropriate slang (e.g., 'courting death', 'brat', 'senior'). Sometimes the words you see are incorrect, so make assumptions when necessary on what the sentence really means. You will output nothing but translations."},
    {"role": "user", "content": f"Translate: {ocr_text}"}
]
tokenized_chat = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True       
).to(model.device)


outputs = model.generate(
    **tokenized_chat, 
    max_new_tokens=2048
)

prompt_length = tokenized_chat.input_ids.shape[1]
new_tokens = outputs[0][prompt_length:]

output_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
print(output_text)

Go! You can do it!

Are you being jealous?

Give me some respect and stop talking about it.

You’re really something… kind of weird.

It’s like inviting death into your own life!

You’re seeking death.
